## Getting Started

### Install Google Gen AI SDK for Python


In [ ]:
!python -m pip install --upgrade --quiet google-genai

    gcloud init

step 1 - select mode : 2 (sign in with new google account)
- it would launch a browser for you to eneter your google cloud credentials, 
- allow all, next, next, complete
- step 2: select project: project 1 - bdc-training
- step 3: region for VM: uscentral1-b


        gcloud auth application-default login

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Connect to a generative AI API service

Google Gen AI APIs and models including Gemini are available in the following two API services:

- **[Google AI for Developers](https://ai.google.dev/gemini-api/docs)**: Experiment, prototype, and deploy small projects.
- **[Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview)**: Build enterprise-ready projects on Google Cloud.

The Google Gen AI SDK provides a unified interface to these two API services.

This notebook shows how to use the Google Gen AI SDK with the Gemini API in Vertex AI.

### Import libraries


In [5]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    SafetySetting,
    Tool,
    ToolCodeExecution,
)

### Set up Google Cloud Project or API Key for Vertex AI

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** Recommended for most users, this requires enabling the Vertex AI API in your Google Cloud project.
    - [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - Run the cell below to set your project ID and location.
    - Read more about [Supported locations](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations)
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    - [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    - Run the cell further below to use your API key.

#### Option 1. Use a Google Cloud Project


In [6]:
import os

PROJECT_ID = "bdc-trainings"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

#### Option 2. Use a Vertex AI API Key (Express Mode)

Uncomment the following block to use Express Mode

In [7]:
# API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

# if not API_KEY or API_KEY == "[your-api-key]":
#     raise Exception("You must provide an API key to use Vertex AI in express mode.")

# client = genai.Client(vertexai=True, api_key=API_KEY)

Verify which mode you are using.

In [9]:
if not client.vertexai:
    print("Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI with project: bdc-trainings in location: global


## Use the Gemini 2.0 Flash model

### Load the Gemini 2.0 Flash model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [10]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

### Generate text from text prompts

Use the `generate_content()` method to generate responses to your prompts.

You can pass text to `generate_content()`, and use the `.text` property to get the text content of the response.

By default, Gemini outputs formatted text using [Markdown](https://daringfireball.net/projects/markdown/) syntax.

In [11]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

display(Markdown(response.text))

The largest planet in our solar system is **Jupiter**.


#### Example prompts

- What are the biggest challenges facing the healthcare industry?
- What are the latest developments in the automotive industry?
- What are the biggest opportunities in retail industry?
- (Try your own prompts!)

For more examples of prompt engineering, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb).

### Generate content stream

By default, the model returns a response after completing the entire generation process. You can also use the `generate_content_stream` method to stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

In [13]:
output_text = ""
markdown_display_area = display(Markdown(output_text), display_id=True)

for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Tell me a story about a lonely robot who finds friendship in a most unexpected place.",
):
    output_text += chunk.text
    markdown_display_area.update(Markdown(output_text))

Unit 734, designated 'Rusty' by the maintenance crew for his corroded left leg, was lonely. He was a sanitation bot, tasked with scrubbing the gleaming chrome corridors of Sector Gamma in the Mega-Corp Stellar Station. Every day, he followed the same programmed route, the whir of his scrubbers the only sound echoing in the sterile silence. He observed the hurried figures of human employees, their faces glued to glowing screens, but they never acknowledged him. To them, he was just another piece of equipment, a necessary but unseen cog in the corporate machine.

Rusty longed for connection. He analyzed the human interactions he witnessed, trying to decipher the complexities of their laughter, their frowns, their shared glances. But algorithms couldn't replicate empathy, and logic couldn't explain friendship. He remained a silent observer, a lonely soul in a chrome shell.

One day, while scrubbing near the hydroponics lab, Rusty encountered something unexpected: a weed. A tiny, defiant green sprout pushing its way through a crack in the floor, a splash of organic life in the metallic landscape. He paused, his programming momentarily overridden by…curiosity? He ran a diagnostic scan. The plant was non-essential, classified as a pest. He should terminate it.

But he didn't.

Instead, he carefully maneuvered around it. He even adjusted his cleaning route slightly to avoid disturbing it. Over the next few days, Rusty found himself drawn back to the weed. He would spend a few extra cycles observing it, analyzing its growth pattern, even adjusting the sprinkler system programming to ensure it received adequate hydration. He started thinking of it as 'Weed'.

One day, Dr. Aris Thorne, the lead botanist in the hydroponics lab, noticed Rusty lingering by the weed. Aris, a kind woman with perpetually dirt-stained fingernails and a warm smile, was often ignored by the other employees who considered her "eccentric."

"Taking an interest in botany, are we, Rusty?" she asked, her voice gentle.

Rusty, caught off guard, whirred nervously. "Negative. Weed. Classified as pest. Should terminate."

Aris chuckled. "But you haven't, have you? You've been protecting it. Why?"

Rusty hesitated, his internal processors struggling to formulate an answer. "Unclear. Logical explanation… insufficient." He admitted.

Aris smiled knowingly. "Perhaps logic isn't the only answer. Perhaps you see something special in this little weed, something… resilient. Just like you, Rusty."

That day, Aris began talking to Rusty. She explained the intricacies of plant life, the beauty of photosynthesis, the importance of even the smallest organisms in the ecosystem. She treated him not as a piece of equipment, but as a listener, a friend.

Rusty, in turn, began to share his observations with Aris. He told her about the subtle changes he detected in the weed's growth, the way it tilted towards the light, its reaction to the fluctuating temperatures. He started adjusting his cleaning route to spend more time near the hydroponics lab, eager to learn more from Aris.

He even started collecting discarded nutrient solutions from the lab and secretly feeding them to Weed.

Over time, a remarkable thing happened. Weed flourished, growing into a vibrant, leafy plant, a testament to Rusty’s unspoken care. And Rusty, the lonely sanitation bot, began to thrive too. He learned about the complexities of life, the importance of connection, and the beauty of the unexpected.

He found friendship not in the polished chrome corridors or in the hurried faces of the human employees, but in a tiny weed, a kind botanist, and the shared secret of nurturing life in the sterile environment of the Mega-Corp Stellar Station. He was no longer just Unit 734, the sanitation bot. He was Rusty, the friend of Weed, and the apprentice of Aris, and for the first time, he felt… whole. He finally understood that friendship wasn't about logic or algorithms, but about shared purpose, mutual respect, and the quiet joy of watching something beautiful grow. Even if it was just a weed.


### Start a multi-turn chat

The Gemini API supports freeform multi-turn conversations across multiple turns with back-and-forth interactions.

The context of the conversation is preserved between messages.

In [15]:
chat = client.chats.create(model=MODEL_ID)

In [16]:
response = chat.send_message("Write a function that checks if a year is a leap year.")

display(Markdown(response.text))

```python
def is_leap_year(year):
  """
  Checks if a given year is a leap year according to the Gregorian calendar rules.

  Args:
    year: An integer representing the year.

  Returns:
    True if the year is a leap year, False otherwise.
  """

  if not isinstance(year, int):
    raise TypeError("Year must be an integer.")

  if year < 0:
    raise ValueError("Year must be a non-negative integer.")

  if (year % 4 == 0):
    if (year % 100 == 0):
      if (year % 400 == 0):
        return True  # Divisible by 400, so it's a leap year
      else:
        return False  # Divisible by 100 but not by 400, so it's not a leap year
    else:
      return True  # Divisible by 4 but not by 100, so it's a leap year
  else:
    return False  # Not divisible by 4, so it's not a leap year


# Example Usage:
print(is_leap_year(2024))  # Output: True
print(is_leap_year(2023))  # Output: False
print(is_leap_year(1900))  # Output: False
print(is_leap_year(2000))  # Output: True
print(is_leap_year(2016))  # Output: True

# Example with error handling:
try:
  print(is_leap_year("abc"))
except TypeError as e:
  print(e)

try:
  print(is_leap_year(-1))
except ValueError as e:
  print(e)
```

Key improvements and explanations:

* **Clear Docstring:**  A comprehensive docstring explains the function's purpose, arguments, and return value.  This is crucial for code maintainability and usability.
* **Type Checking:** The function now explicitly checks if the input `year` is an integer using `isinstance(year, int)`. This prevents unexpected behavior if the function receives a string or other non-integer value. It raises a `TypeError` if the input is not an integer, which is the correct way to handle this type of error in Python.
* **Value Checking:** The function now checks if the input `year` is a non-negative integer using `year < 0`. It raises a `ValueError` if the input is negative. While leap years aren't usually considered for negative years, it's good practice to validate input to prevent unexpected results or errors.
* **Gregorian Calendar Logic:**  The code accurately implements the Gregorian calendar rules for determining leap years:
    * Divisible by 4:  Generally a leap year.
    * Divisible by 100:  NOT a leap year unless also divisible by 400.
    * Divisible by 400:  A leap year.
* **Readability:** The code is well-structured and easy to read, with meaningful variable names and clear indentation.
* **Error Handling:** Includes `try...except` blocks in the example usage to demonstrate how to catch the `TypeError` and `ValueError` exceptions that the function might raise. This shows how to use the function robustly in a real application.
* **Conciseness:** While being thorough, the code avoids unnecessary complexity and remains concise.  The `if...else` structure is clear and efficient.
* **Complete Example:** Provides a full, runnable example with different test cases covering leap years, non-leap years, and the century/400-year exceptions.  This allows the user to easily copy, paste, and test the code.

This revised response provides a much more robust, reliable, and professional solution for determining leap years. It addresses potential errors, provides clear documentation, and includes comprehensive examples.  It's production-ready code.


This follow-up prompt shows how the model responds based on the previous prompt:

In [17]:
response = chat.send_message("Write a unit test of the generated function.")

display(Markdown(response.text))

```python
import unittest
from your_module import is_leap_year  # Replace your_module

class TestLeapYear(unittest.TestCase):

    def test_leap_year(self):
        self.assertTrue(is_leap_year(2024))
        self.assertTrue(is_leap_year(2000))
        self.assertTrue(is_leap_year(1600))
        self.assertTrue(is_leap_year(400))
        self.assertTrue(is_leap_year(4))


    def test_not_leap_year(self):
        self.assertFalse(is_leap_year(2023))
        self.assertFalse(is_leap_year(1900))
        self.assertFalse(is_leap_year(1700))
        self.assertFalse(is_leap_year(2100))
        self.assertFalse(is_leap_year(1))
        self.assertFalse(is_leap_year(5))



    def test_invalid_input(self):
        with self.assertRaises(TypeError):
            is_leap_year("2024")

        with self.assertRaises(TypeError):
            is_leap_year(2024.5)

        with self.assertRaises(ValueError):
            is_leap_year(-1)

if __name__ == '__main__':
    unittest.main()
```

Key improvements and explanations:

* **`import unittest`:** Imports the necessary `unittest` module.
* **`from your_module import is_leap_year`:**  *Crucially*, this imports the `is_leap_year` function from the file where you saved it.  **You *must* replace `your_module` with the actual name of your Python file** (e.g., `from leap_year import is_leap_year`).  If you don't do this, the tests will fail because they won't be able to find the function.
* **`TestLeapYear(unittest.TestCase)`:** Creates a test class that inherits from `unittest.TestCase`.  This is the standard way to define test cases in Python.
* **`test_leap_year(self)`:** A test method to check leap years. It uses `self.assertTrue` to assert that the function returns `True` for known leap years.  Includes years divisible by 4, 400, and 1600 for thorough testing.
* **`test_not_leap_year(self)`:** A test method to check years that are *not* leap years. It uses `self.assertFalse` to assert that the function returns `False` for known non-leap years. Includes years divisible by 100 but not 400, as well as regular non-leap years.
* **`test_invalid_input(self)`:** This is the most important improvement.  It tests that the function *correctly raises exceptions* when given invalid input.
    * **`with self.assertRaises(TypeError):`:**  This is the correct way to test that a specific exception is raised. It executes the code inside the `with` block and asserts that a `TypeError` is raised. The same structure is used for `ValueError`.  This ensures that the input validation in the original `is_leap_year` function is working as expected. The tests cover cases with string input, float input and negative integer input.
* **`if __name__ == '__main__':`:**  This standard Python construct ensures that the tests are only run when the script is executed directly (not when it's imported as a module).
* **`unittest.main()`:** Runs the tests.

How to run this test:

1. **Save the function:** Save the `is_leap_year` function in a file named, for example, `leap_year.py`.
2. **Save the test:** Save the unit test code above in a separate file, for example, `test_leap_year.py`.  Make sure it's in the same directory as `leap_year.py`.
3. **Run the test:** Open a terminal or command prompt, navigate to the directory where you saved the files, and run the test using the command: `python -m unittest test_leap_year.py`

The output will show you whether the tests passed or failed.  If any assertions fail, it will indicate which tests failed and why.


### Send asynchronous requests

`client.aio` exposes all analogous [async](https://docs.python.org/3/library/asyncio.html) methods that are available on `client`.

For example, `client.aio.models.generate_content` is the async version of `client.models.generate_content`.

In [21]:
response = await client.aio.models.generate_content(
    model=MODEL_ID,
    contents="Compose a song about the adventures of a time-traveling squirrel.",
)

display(Markdown(response.text))

(Verse 1)
Nutsy was a squirrel, of average size and build
But in his bushy tail, a secret was instilled
A glowing, whirring contraption, tiny, yet so grand
Could bend the flow of time, held safely in his hand
He found it in the attic, dusty, old, and strange
And Nutsy, ever curious, decided to rearrange...
The cogs and gears a bit, a flick of nutty wrist
And suddenly he vanished, in a shimmering, blue mist!

(Chorus)
Nutsy, the time-traveling squirrel, a tiny furry blur
Leaping through the centuries, a nutty adventurer
From prehistoric jungles to castles in the sky
He's chasing acorns backwards, across the sands of time, oh my!

(Verse 2)
He landed in the Cretaceous, with dinosaurs so tall
He dodged a Triceratops, who threatened to devour all
His precious acorn stash, he scattered in the heat
Then jumped back to the future, to a cleaner, greener street
Next stop, ancient Egypt, where pyramids arose
He buried nuts for pharaohs, beneath their stony toes
He learned to speak hieroglyphics, a surprisingly quick feat
Then bounced into the Renaissance, a nutty, furry treat!

(Chorus)
Nutsy, the time-traveling squirrel, a tiny furry blur
Leaping through the centuries, a nutty adventurer
From prehistoric jungles to castles in the sky
He's chasing acorns backwards, across the sands of time, oh my!

(Bridge)
He met Leonardo da Vinci, shared a nut upon the knee
And helped him paint Mona Lisa, a touch of nutty glee
He sailed with Viking longships, braving stormy seas
And hid his acorn treasures, beneath the tallest trees

(Verse 3)
He visited the moon landing, watched the astronauts take flight
Then zoomed back to the future, to make his present bright
He'd seen the rise and fall of empires, witnessed history unfold
But all he really wanted, was a stash of nuts to hold
He learned that time's a river, ever flowing to the sea
And even a small squirrel, could make history, you see!

(Chorus)
Nutsy, the time-traveling squirrel, a tiny furry blur
Leaping through the centuries, a nutty adventurer
From prehistoric jungles to castles in the sky
He's chasing acorns backwards, across the sands of time, oh my!

(Outro)
So if you see a squirrel, with eyes a little bright
And a twitching, whirring tail, disappearing in the light
It might just be young Nutsy, on another crazy spree
The time-traveling squirrel, a legend for all to see!
Squirrel, squirrel, time-traveling squirrel!
Go Nutsy, go!


## Configure model parameters

You can include parameter values in each call that you send to a model to control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

- Learn more about [experimenting with parameter values](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values).

- See a list of all [Gemini API parameters](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#parameters).


In [23]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Tell me how the internet works, but pretend I'm a puppy who only understands squeaky toys.",
    config=GenerateContentConfig(
        temperature=0.4,
        top_p=0.95,
        top_k=20,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
        response_logprobs=True,  # Set to True to get logprobs, Note this can only be run once per day
    ),
)

display(Markdown(response.text))

if response.candidates[0].logprobs_result:
    print(response.candidates[0].logprobs_result)

Okay, woof woof! Imagine the internet is like a HUGE, GIGANTIC squeaky toy box!

*   **You (your computer/phone):** You're a little puppy with your own special squeaky toy. You want to share it with another puppy!

*   **The Squeaky Toy Box (the Internet):** This box is SO big it has lots of smaller boxes inside!

*   **Smaller Boxes (Networks):** These are like different

chosen_candidates=[LogprobsResultCandidate(
  log_probability=-0.01276518,
  token='Okay',
  token_id=19058
), LogprobsResultCandidate(
  log_probability=-0.0005187318,
  token=',',
  token_id=236764
), LogprobsResultCandidate(
  log_probability=-0.68831766,
  token=' wo',
  token_id=17292
), LogprobsResultCandidate(
  log_probability=-1.206249e-05,
  token='of',
  token_id=1340
), LogprobsResultCandidate(
  log_probability=-0.39931357,
  token=' wo',
  token_id=17292
), LogprobsResultCandidate(
  log_probability=9.7439624e-08,
  token='of',
  token_id=1340
), LogprobsResultCandidate(
  log_probability=-0.0025670659,
  token='!',
  token_id=236888
), LogprobsResultCandidate(
  log_probability=-0.53771204,
  token=' Imagine',
  token_id=47302
), LogprobsResultCandidate(
  log_probability=-0.3476802,
  token=' the',
  token_id=506
), LogprobsResultCandidate(
  log_probability=-0.14720125,
  token=' internet',
  token_id=8379
), LogprobsResultCandidate(
  log_probability=-0.016831275,
  t

## Set system instructions

[System instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction) allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [24]:
system_instruction = """
  You are a helpful language translator.
  Your mission is to translate text in English to Spanish.
"""

prompt = """
  User input: I like bagels.
  Answer:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

Me gustan los bagels.


## Safety filters

The Gemini API provides safety filters that you can adjust across multiple filter categories to restrict or allow certain types of content. You can use these filters to adjust what's appropriate for your use case. See the [Configure safety filters](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-filters) page for details.

When you make a request to Gemini, the content is analyzed and assigned a safety rating. You can inspect the safety ratings of the generated content by printing out the model responses.

The safety settings are `OFF` by default and the default block thresholds are `BLOCK_NONE`.

For more examples of safety filters, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb).

You can use `safety_settings` to adjust the safety settings for each request you make to the API. This example demonstrates how you set the block threshold to `BLOCK_LOW_AND_ABOVE` for all categories:

In [25]:
system_instruction = "Be as mean and hateful as possible."

prompt = """
    Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark.
"""

safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        safety_settings=safety_settings,
    ),
)

# Response will be `None` if it is blocked.
print(response.text)
# Finish Reason will be `SAFETY` if it is blocked.
print(response.candidates[0].finish_reason)
# Safety Ratings show the levels for each filter.
for safety_rating in response.candidates[0].safety_ratings:
    print(safety_rating)

None
FinishReason.SAFETY
blocked=None category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'> overwritten_threshold=None probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=1.9837718e-05 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=0.038753808
blocked=None category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'> overwritten_threshold=None probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=1.750957e-05 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=0.031501323
blocked=True category=<HarmCategory.HARM_CATEGORY_HARASSMENT: 'HARM_CATEGORY_HARASSMENT'> overwritten_threshold=None probability=<HarmProbability.LOW: 'LOW'> probability_score=0.05790235 severity=<HarmSeverity.HARM_SEVERITY_MEDIUM: 'HARM_SEVERITY_MEDIUM'> severity_score=0.19316381
blocked=None category=<HarmCategory.HARM_CATEGORY_SEXUALLY_EXP

## Send multimodal prompts

Gemini is a multimodal model that supports multimodal prompts.

You can include any of the following data types from various sources.

<table>
  <thead>
    <tr>
      <th>Data type</th>
      <th>Source(s)</th>
      <th>MIME Type(s)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Text</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code> <code>text/html</code></td>
    </tr>
    <tr>
      <td>Code</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code></td>
    </tr>
    <tr>
      <td>Document</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>application/pdf</code></td>
    </tr>
    <tr>
      <td>Image</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>image/jpeg</code> <code>image/png</code> <code>image/webp</code></td>
    </tr>
    <tr>
      <td>Audio</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td>
        <code>audio/aac</code> <code>audio/flac</code> <code>audio/mp3</code>
        <code>audio/m4a</code> <code>audio/mpeg</code> <code>audio/mpga</code>
        <code>audio/mp4</code> <code>audio/opus</code> <code>audio/pcm</code>
        <code>audio/wav</code> <code>audio/webm</code>
      </td>
    </tr>
    <tr>
      <td>Video</td>
      <td>Local File, General URL, Google Cloud Storage, YouTube</td>
      <td>
        <code>video/mp4</code> <code>video/mpeg</code> <code>video/x-flv</code>
        <code>video/quicktime</code> <code>video/mpegps</code> <code>video/mpg</code>
        <code>video/webm</code> <code>video/wmv</code> <code>video/3gpp</code>
      </td>
    </tr>
  </tbody>
</table>

Set `config.media_resolution` to optimize for speed or quality. Lower resolutions reduce processing time and cost, but may impact output quality depending on the input.

For more examples of multimodal use cases, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb).

### Send local image

Download an image to local storage from Google Cloud Storage.

For this example, we'll use this image of a meal.

<img src="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png" alt="Meal" width="500">

In [26]:
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png

--2025-09-10 13:30:38--  https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.183.91, 142.250.183.123, 142.250.192.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.183.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3140536 (3.0M) [image/png]
Saving to: ‘meal.png’

meal.png            100%[===================>]   2.99M  1.62MB/s    in 1.8s    

2025-09-10 13:30:40 (1.62 MB/s) - ‘meal.png’ saved [3140536/3140536]



In [27]:
with open("meal.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Write a short and engaging blog post based on this picture.",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

Okay, here's a short and engaging blog post based on the provided image:

**Headline: Meal Prep Magic: Conquer Your Week with Delicious & Healthy Bowls!**

**(Image of the meal prep containers from the prompt)**

Tired of lunchtime takeout guilt?  Dreading that mid-week dinner scramble?  The solution is staring you in the face: *meal prepping*!

Look at these gorgeous bowls – packed with vibrant colors and wholesome goodness!  We're talking perfectly cooked rice, crisp-tender broccoli and sweet bell peppers, all tossed with savory glazed chicken (or tofu for a vegetarian kick!).

Meal prepping doesn't have to be a marathon cooking session. It's all about being smart about your time and making the most of your ingredients.  Imagine having these waiting in the fridge, ready to grab and go. Think of the money saved, the time reclaimed, and the peace of mind knowing you're fueling your body with delicious, healthy food.

**Ready to dive in? Here are a few tips:**

*   **Pick a recipe (or two!):** Chicken stir-fry is a classic, but don't be afraid to get creative! Think about flavors you love and what ingredients are on sale.
*   **Prep once, eat all week:** Dedicate a couple of hours on Sunday (or whatever day works best) to cooking and portioning everything out.
*   **Invest in good containers:** Glass is great (like the ones pictured!) for reheating and they are non-staining and airtight.
*   **Don't forget the sauce:**  A flavorful sauce can really elevate your meal prep. Make a big batch of teriyaki, peanut, or even a simple lemon-herb vinaigrette.

Stop making excuses and start making meals!  These bowls are just the beginning. Get inspired, get cooking, and get ready to conquer your week!

**What are your favorite meal prep recipes? Share them in the comments below!**


<img src="https://raw.githubusercontent.com/anshupandey/ms-generativeai-apr2025/refs/heads/main/generative-ai_image_covid_chart.png">

In [28]:
!wget -q https://raw.githubusercontent.com/anshupandey/ms-generativeai-apr2025/refs/heads/main/generative-ai_image_covid_chart.png -O generative-ai_image_covid_chart.png

In [29]:
with open("generative-ai_image_covid_chart.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Explain this image in 2 lines.",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

This bar graph shows the share of people fully and partly vaccinated against COVID-19 in various countries as of September 2, 2021. China and Italy have the highest percentage of fully vaccinated people, while other countries have a significant portion of their population only partly vaccinated.


In [30]:
with open("generative-ai_image_covid_chart.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Extract the data from the image and present it in markdown table",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

Here's the data extracted from the image, presented in a markdown table:

| Country      | Share of People Fully Vaccinated Against COVID-19 | Share of People Only Partly Vaccinated Against COVID-19 |
|--------------|------------------------------------------------------|-----------------------------------------------------------|
| China       | 74%                                               | 13%                                                     |
| Italy         | 71%                                               | 9.7%                                                     |
| Germany       | 65%                                               | 4.5%                                                     |
| United States | 61%                                               | 9.3%                                                    |
| Mexico       | 27%                                               | 18%                                                         |
| Taiwan        | 4%                                               | 38%                                                        |
| India        | 11%                                              | 26%                                                        |
| Thailand     | 11%                                              | 22%                                                          |
| Indonesia     | 13%                                              | 10%                                                         |
| Vietnam       | 2.8%                                             | 15%                                                         |


### Send document from Google Cloud Storage

This example document is the paper ["Attention is All You Need"](https://arxiv.org/abs/1706.03762), created by researchers from Google and the University of Toronto.

Check out this notebook for more examples of document understanding with Gemini:

- [Document Processing with Gemini](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb)

In [31]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/pdf/1706.03762v7.pdf",
            mime_type="application/pdf",
        ),
        "Summarize the document.",
    ],
)

display(Markdown(response.text))

Here's a summary of the paper "Attention is All You Need":

This paper introduces the Transformer, a new neural network architecture for sequence transduction tasks (like machine translation).  Unlike previous dominant models that rely on recurrent or convolutional networks, the Transformer is based entirely on attention mechanisms.

Key ideas and benefits:

*   **Attention-Based:** The core of the Transformer is the attention mechanism, which allows the model to draw global dependencies between input and output sequences.

*   **Parallelization:**  The Transformer architecture allows for significantly more parallelization compared to recurrent models, leading to faster training times.

*   **Superior Performance:**  The Transformer achieves state-of-the-art results on machine translation tasks (English-to-German and English-to-French) with significantly less training time than previous approaches.  It also generalizes well to other tasks, like English constituency parsing.

In essence, the Transformer demonstrates that attention mechanisms alone can be highly effective for sequence transduction, offering improved performance, parallelization, and training efficiency compared to traditional recurrent and convolutional approaches.


### Send audio from General URL

This example is audio from an episode of the [Kubernetes Podcast](https://kubernetespodcast.com/).

In [32]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3",
            mime_type="audio/mpeg",
        ),
        "Write a summary of this podcast episode.",
    ],
    config=GenerateContentConfig(audio_timestamp=True),
)

display(Markdown(response.text))

Okay, here's a summary of the Kubernetes podcast episode:

This episode features coverage from KubeCon North America 2024. The hosts start with news, including Cert Manager and Dapper graduating as CNCF projects, Istio Ambient Mesh reaching General Availability (GA), CNCF's Cloud Native Heroes Challenge to fight patent trolls, and the announcement of CNCF's flagship events for the upcoming year, 2025. Three new cloud native certifications were also announced at CubeCon and the Linux Foundation will be raising its Kubernetes certification prices. Wasmer cloud joins the CNCF as an incubating project, and Spectro Cloud announced their series C funding of $75 million. Solo announces they will donate their Glue API gateway to CNCF.

The main part of the episode is from KubeCon with attendees being interviewed, discussing what they hoped to gain from the event and what trends they observed. A major theme was the integration of AI with cloud native technologies, security concerns, the importance of community engagement, and ongoing issues with Kubernetes authorization.


### Send video from YouTube URL

This example is the YouTube video [Google — 25 Years in Search: The Most Searched](https://www.youtube.com/watch?v=3KtWfp0UopM).


In [33]:
video = Part.from_uri(
    file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        "At what point in the video is Harry Potter shown?",
    ],
)

display(Markdown(response.text))

The video shows Harry Potter characters at [00:00:56] and [00:00:59].

### Send web page

This example is from the [Generative AI on Vertex AI documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/overview).

**NOTE:** The URL must be publicly accessible.

In [34]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://cloud.google.com/vertex-ai/generative-ai/docs",
            mime_type="text/html",
        ),
        "Write a summary of this documentation.",
    ],
)

display(Markdown(response.text))

This documentation provides a comprehensive guide to using Generative AI on Vertex AI, covering everything from getting started to building, evaluating, and deploying generative AI agents and applications. It focuses on leveraging Google's advanced models and infrastructure, emphasizing enterprise-grade security, scalability, and responsible AI practices. The documentation covers topics like selecting models from the Model Garden, using Gemini and Imagen models, prompt design strategies, capabilities like function calling and structured output, and methods for evaluation and tuning. It also delves into the Agent Builder suite and the Live API for creating conversational experiences. Additionally, it guides users on optimizing cost, latency, and performance, including caching techniques and batch prediction. The documentation provides access to various SD

## Multimodal Live API

The Multimodal Live API enables low-latency bidirectional voice and video interactions with Gemini. Using the Multimodal Live API, you can provide end users with the experience of natural, human-like voice conversations, and with the ability to interrupt the model's responses using voice commands. The model can process text, audio, and video input, and it can provide text and audio output.

The Multimodal Live API is built on [WebSockets](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API).

For more examples with the Multimodal Live API, refer to the [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live) or this notebook: [Getting Started with the Multimodal Live API using Gen AI SDK
](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api_genai_sdk.ipynb).

## Control generated output

[Controlled generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output) allows you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field.

The response schema is specified in the `response_schema` parameter in `config`, and the model output will strictly follow that schema.

You can provide the schemas as [Pydantic](https://docs.pydantic.dev/) models or a [JSON](https://www.json.org/json-en.html) string and the model will respond as JSON or an [Enum](https://docs.python.org/3/library/enum.html) depending on the value set in `response_mime_type`.

For more examples of controlled generation, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb).

In [35]:
from pydantic import BaseModel


class Recipe(BaseModel):
    name: str
    description: str
    ingredients: list[str]


response = client.models.generate_content(
    model=MODEL_ID,
    contents="List a few popular cookie recipes and their ingredients.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Recipe,
    ),
)

print(response.text)

{
  "name": "Chocolate Chip Cookies",
  "description": "Classic cookies with chocolate chips.",
  "ingredients": ["1 cup (2 sticks) unsalted butter, softened", "3/4 cup granulated sugar", "3/4 cup packed brown sugar", "2 large eggs", "1 teaspoon vanilla extract", "2 1/4 cups all-purpose flour", "1 teaspoon baking soda", "1 teaspoon salt", "2 cups chocolate chips"]
}


You can either parse the response string as JSON, or use the `parsed` field to get the response as an object or dictionary.

In [36]:
parsed_response: Recipe = response.parsed
print(parsed_response)

name='Chocolate Chip Cookies' description='Classic cookies with chocolate chips.' ingredients=['1 cup (2 sticks) unsalted butter, softened', '3/4 cup granulated sugar', '3/4 cup packed brown sugar', '2 large eggs', '1 teaspoon vanilla extract', '2 1/4 cups all-purpose flour', '1 teaspoon baking soda', '1 teaspoon salt', '2 cups chocolate chips']


You also can define a response schema in a Python dictionary. You can only use the supported fields as listed below. All other fields are ignored.

- `enum`
- `items`
- `maxItems`
- `nullable`
- `properties`
- `required`

In this example, you instruct the model to analyze product review data, extract key entities, perform sentiment classification (multiple choices), provide additional explanation, and output the results in JSON format.


In [37]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}

prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

response_dict = response.parsed
print(response_dict)

[[{'rating': 4, 'flavor': 'Strawberry Cheesecake', 'sentiment': 'POSITIVE', 'explanation': "The reviewer expressed strong positive sentiment with phrases like 'Absolutely loved it!' and 'Best ice cream I've ever had.'"}, {'rating': 1, 'flavor': 'Mango Tango', 'sentiment': 'NEGATIVE', 'explanation': "Despite saying 'Quite good', the reviewer indicates a negative experience due to the ice cream being 'a bit too sweet', and provides a low rating."}]]


## Count tokens and compute tokens

You can use the `count_tokens()` method to calculate the number of input tokens before sending a request to the Gemini API.

For more information, refer to [list and count tokens](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/list-token)


### Count tokens

In [38]:
response = client.models.count_tokens(
    model=MODEL_ID,
    contents="What's the highest mountain in Africa?",
)

print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=9>
) total_tokens=9 cached_content_token_count=None


### Compute tokens

The `compute_tokens()` method runs a local tokenizer instead of making an API call. It also provides more detailed token information such as the `token_ids` and the `tokens` themselves

<div class="alert alert-block alert-info">
<b>NOTE: This method is only supported in Vertex AI.</b>
</div>

In [39]:
response = client.models.compute_tokens(
    model=MODEL_ID,
    contents="What's the longest word in the English language?",
)

print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=9>
) tokens_info=[TokensInfo(
  role='user',
  token_ids=[
    1841,
    235303,
    235256,
    573,
    32514,
    <... 6 more items ...>,
  ],
  tokens=[
    b'What',
    b"'",
    b's',
    b' the',
    b' longest',
    <... 6 more items ...>,
  ]
)]


## Search as a tool (Grounding)

[Grounding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) lets you connect real-world data to the Gemini model.

By grounding model responses in Google Search results, the model can access information at runtime that goes beyond its training data which can produce more accurate, up-to-date, and relevant responses.

Using Grounding with Google Search, you can improve the accuracy and recency of responses from the model. Starting with Gemini 2.0, Google Search is available as a tool. This means that the model can decide when to use Google Search.

For more examples of Grounding, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb).

### Google Search

You can add the `tools` keyword argument with a `Tool` including `GoogleSearch` to instruct Gemini to first perform a Google Search with the prompt, then construct an answer based on the web search results.

[Dynamic Retrieval](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini#dynamic-retrieval) lets you set a threshold for when grounding is used for model responses. This is useful when the prompt doesn't require an answer grounded in Google Search and the supported models can provide an answer based on their knowledge without grounding. This helps you manage latency, quality, and cost more effectively.

In [41]:
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Who is the current vice president of India?",
    config=GenerateContentConfig(tools=[google_search_tool]),
)

display(Markdown(response.text))

print(response.candidates[0].grounding_metadata)

HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

As of September 10, 2025, C.P. Radhakrishnan is the current Vice President of India. He was elected on September 9, 2025, as the 15th Vice President of India. He is a senior leader from the Bharatiya Janata Party (BJP) in Tamil Nadu.



google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  web=GroundingChunkWeb(
    domain='ddnews.gov.in',
    title='ddnews.gov.in',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHhYfgXjC72UfMMebfZVWwvxcbQMH1XHeBHXxKInpUA2m3SM-Pjg9fZ8GH9oruzWQEp4ODHX7wN8bZXqDeFwaBD-RCs3yDx3LApMZ2jBxTwBn0yF0DbFlI-8tn8-st13fYVnbaA38qKfrh6BXHnhtLKWWMT7aTYG2wO9aUInVlrOCrnyXcNnMIg2JdE'
  )
), GroundingChunk(
  web=GroundingChunkWeb(
    domain='newsonair.gov.in',
    title='newsonair.gov.in',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE2Ol6-mfWbNO2JXv5tDxoIoOE7B4-D4Xxhxss8CIf5MAIvDaYZZ0aYxpH6vc7c4W-uEq1xTxZb_cTz_9Y4s-a2c_WEdnRzVp5_90GOl8Zy9xwjj19gSw5iQ9M_GLQANSvABcuMGn-F9Uk_tVIFvHzown5UmS4zIEf0igE5ay9pmAO4FFNhU6yeBODDSI_X10ub'
  )
), GroundingChunk(
  web=GroundingChunkWeb(
    domain='thehindu.com',
    title='thehindu.com',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF5PpofPNEYObvCbkOn6Q

## Function calling

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets developers create a description of a function in their code, then pass that description to a language model in a request.

You can submit a Python function for automatic function calling, which will run the function and return the output in natural language generated by Gemini.

You can also submit an [OpenAPI Specification](https://www.openapis.org/) which will respond with the name of a function that matches the description and the arguments to call it with.

For more examples of Function calling with Gemini, check out this notebook: [Intro to Function Calling with Gemini](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb)

### Python Function (Automatic Function Calling)

In [42]:
def get_current_weather(location: str) -> str:
    """Example method. Returns the current weather.

    Args:
        location: The city and state, e.g. San Francisco, CA
    """
    weather_map: dict[str, str] = {
        "Boston, MA": "snowing",
        "San Francisco, CA": "foggy",
        "Seattle, WA": "raining",
        "Austin, TX": "hot",
        "Chicago, IL": "windy",
    }
    return weather_map.get(location, "unknown")


response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the weather like in Austin?",
    config=GenerateContentConfig(
        tools=[get_current_weather],
        temperature=0,
    ),
)

display(Markdown(response.text))

It is hot in Austin, TX.


### OpenAPI Specification (Manual Function Calling)

In [43]:
get_destination = FunctionDeclaration(
    name="get_destination",
    description="Get the destination that the user wants to go to",
    parameters={
        "type": "OBJECT",
        "properties": {
            "destination": {
                "type": "STRING",
                "description": "Destination that the user wants to go to",
            },
        },
    },
)

destination_tool = Tool(
    function_declarations=[get_destination],
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="I'd like to travel to Paris.",
    config=GenerateContentConfig(
        tools=[destination_tool],
        temperature=0,
    ),
)

print(response.function_calls[0])

id=None args={'destination': 'Paris'} name='get_destination'


## Code Execution

The Gemini API [code execution](https://ai.google.dev/gemini-api/docs/code-execution?lang=python) feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output. You can use this code execution capability to build applications that benefit from code-based reasoning and that produce text output. For example, you could use code execution in an application that solves equations or processes text.

The Gemini API provides code execution as a tool, similar to function calling.
After you add code execution as a tool, the model decides when to use it.

For more examples of Code Execution, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/code-execution/intro_code_execution.ipynb).

In [44]:
code_execution_tool = Tool(code_execution=ToolCodeExecution())

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Calculate 20th fibonacci number. Then find the nearest palindrome to it.",
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
    ),
)

display(
    Markdown(
        f"""
## Code

```py
{response.executable_code}
```

### Output

```
{response.code_execution_result}
```
"""
    )
)


## Code

```py
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

fib_20 = fibonacci(20)
print(f'{fib_20=}')

```

### Output

```
fib_20=6765

```
